In [4]:
from pathlib import Path
from IPython.display import Markdown
import numpy as np

model_path = Path("./fp32_frezon.pb")
ir_path = Path(model_path).with_suffix(".xml")

In [6]:
#1 image of 224x224 RGB
mo_command = f"""mo
                 --input_model "{model_path}"
                 --input_shape "[1,224,224,3]" 
                 --mean_values="[127.5,127.5,127.5]"
                 --scale_values="[127.5]"
                 --data_type FP16
                 --output_dir "{model_path.parent}"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert TensorFlow to OpenVINO:")
display(Markdown(f"`{mo_command}`"))
if not ir_path.exists():
    print("Exporting TensorFlow model to IR... This may take a few minutes.")
    ! $mo_command
else:
    print(f"IR model {ir_path} already exists.")

Model Optimizer command to convert TensorFlow to OpenVINO:


`mo --input_model "fp32_frezon.pb" --input_shape "[1,224,224,3]" --mean_values="[127.5,127.5,127.5]" --scale_values="[127.5]" --data_type FP16 --output_dir "."`

Exporting TensorFlow model to IR... This may take a few minutes.
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/latest/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: C:\Users\fp32_frezon.xml
[ SUCCESS ] BIN file: C:\Users\fp32_frezon.bin


[ WARNING ]  Use of deprecated cli option --data_type detected. Option use in the following releases will be fatal. 
C:\Users\realmeid\Anaconda3\envs\py37\lib\site-packages\numpy\lib\function_base.py:804: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


In [6]:
from openvino.runtime import Core, Dimension

ie = Core()
model = ie.read_model(model=ir_path, weights=ir_path.with_suffix(".bin"))
compiled_model = ie.compile_model(model=model, device_name="MULTI:CPU,GPU") #Full load balance between devices. HETERO:CPU,GPU different hardware units

input_key = compiled_model.input(0)

#input_key =model.input(0)
#shape = input_key.partial_shape
#shape[2] = -1 #unbounded dynamic shape
#model.reshape({input_key: shape})
#compiled_model = ie.compile_model(model=model, device_name="MULTI:CPU,GPU") #Full load balance between devices. HETERO:CPU,GPU different hardware units
output_key = compiled_model.output(0)
network_input_shape = input_key.shape

In [7]:
input_key


<Shape: [1,224,224,3]>

In [4]:
import cv2

image_filename = #"data/coco_hollywood.jpg"
image = cv2.imread(image_filename)
image.shape

N, C, H, W = input_key.shape
# OpenCV resize expects the destination size as (width, height).
resized_image = cv2.resize(src=image, dsize=(W, H))
resized_image.shape

input_data = np.expand_dims(np.transpose(resized_image, (2, 0, 1)), 0).astype(np.float32)
input_data.shape


ImportError: cannot import name 'IEPlugin' from 'openvino.inference_engine' (C:\Users\realmeid\Anaconda3\envs\py37\lib\site-packages\openvino\inference_engine\__init__.py)

In [ ]:
result = compiled_model([input_data])[output_key]

request = compiled_model.create_infer_request()
request.infer(inputs={input_key.any_name: input_data})
result = request.get_output_tensor(output_key.index).data